# Collaborative Filtering with SVD
Author: Shiyi Wang

In [1]:
import pandas as pd
import numpy as np
import pickle
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate,train_test_split,KFold
from surprise import SVD,SVDpp, NMF,SlopeOne,CoClustering
from collections import defaultdict

In [2]:
df = pd.read_pickle('../data/processed_data.pkl')
df

,rating,name,user_id,recipe_id
0,4,white bean green chile pepper soup,3787,16642
1,5,white bean green chile pepper soup,95286,16642
3,5,baked potato toppings,14502,34897
4,5,baked potato toppings,6559,34897
5,4,sugared raspberries,5690,49598
...,...,...,...,...
1132360,5,easy microwave hot fudge topping,215681,33795
1132361,0,garlic clove chicken,11621,22415
1132364,5,pot roast with port stove top,17831,32160
1132365,4,pot roast with port stove top,5947,32160


Load data using surprise framework

In [3]:
df_copy = df.drop(['name'], axis=1)
df_copy

,rating,user_id,recipe_id
0,4,3787,16642
1,5,95286,16642
3,5,14502,34897
4,5,6559,34897
5,4,5690,49598
...,...,...,...
1132360,5,215681,33795
1132361,0,11621,22415
1132364,5,17831,32160
1132365,4,5947,32160


In [4]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df_copy[['user_id', 'recipe_id', 'rating']], reader)

In [5]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

Use SVD model

In [6]:
algo = SVD(verbose=True)

In [7]:
def kfold_train_test(data,algo,kfold = 5,k = 10,treshold = 3.5):
    kf = KFold(n_splits=kfold)
    precision_kfold = []
    recall_kfold = []

    for trainset, testset in kf.split(data):
        algo.fit(trainset)
        predictions = algo.test(testset)
        precisions, recalls = precision_recall_at_k(predictions, k=k, threshold=treshold)

        # Precision and recall can then be averaged over all users
        precision_kfold.append(sum(prec for prec in precisions.values()) / len(precisions))
        recall_kfold.append(sum(rec for rec in recalls.values()) / len(recalls))
        
    return precision_kfold,recall_kfold

In [8]:
kf = KFold(n_splits=5)
precision_kfold = []
recall_kfold = []

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=3.5)

    # Precision and recall can then be averaged over all users
    precision_kfold.append(sum(prec for prec in precisions.values()) / len(precisions))
    recall_kfold.append(sum(rec for rec in recalls.values()) / len(recalls))

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing

In [9]:
print("5-fold precision@10 is {:.3f}".format(np.mean(precision_kfold)),"\n","5-fold recall@10 is {:.3f}".format(np.mean(recall_kfold)))

5-fold precision@10 is 0.816 
 5-fold recall@10 is 0.963


In [10]:
def save_file_to_pickle(item, file_name, file_type = 'obj'):
    file = open(f'../data/{file_name}.{file_type}', 'wb') 
    pickle.dump(item, file)
    file.close()

In [11]:
save_file_to_pickle(algo,"SVD_algo",'pkl')

In [12]:
uid = 3787
iids = [16642, 5840, 16580, 13811]
recipes_names = dict([(rep_id, name)
                     for name, rep_id in zip(df.name, df.recipe_id)])


In [13]:
def refactorRecipeNames(text):
    text = text.replace(" s ", "\'s ")
    text_split = text.split(" ")
    text_split = [t.strip().capitalize() for t in text_split if t != '']
    return " ".join(text_split)


In [14]:
[refactorRecipeNames(recipes_names[r]) for r in iids]

['White Bean Green Chile Pepper Soup',
 'Black Beans With Mango Sauce',
 'Warm Spinach Salad',
 'Roasted Asparagus Shiitake Mushrooms']

In [15]:
save_file_to_pickle(recipes_names,"recipes_names",'pkl')

In [16]:
def get_n_predictions(iids, algo, n=10, uid=3787, item_name=recipes_names):

    # create the list to search in
    iid_to_test = [iid for iid in range(139684) if iid not in iids]
    # build data for surprise
    test_set = [[uid, iid, 4.] for iid in iid_to_test]
    # predict
    predictions = algo.test(test_set)
    # get prediction
    pred_ratings = [pred.est for pred in predictions]
    # return top_n indexes
    top_n = np.argpartition(pred_ratings, 1)[-n:]
    # return list of recipe names
    results = [item_name[k] for k in top_n]

    return [refactorRecipeNames(r) for r in results]


In [17]:
get_n_predictions(iids, algo)

['Outback Copycat Alice Springs Chicken',
 'Ensalada Criolla',
 'Amish Triple Butter Biscuits',
 'One Pot Chicken Bacon Spinach Parmesan Pasta',
 'Thai Peanut Coconut Chicken',
 'Banana Coffee Cake',
 "Nif's Cajun Macaroni And Cheese",
 'Coco Oatmeal Honey Cookies',
 'Creole Watermelon Feta Salad',
 'Grilled Cajun Green Beans']